<h1> NDVI calculation for Hadrian's Wall 

<h3> Set up notebook 

In [6]:
import ee
import geemap
import os
import numpy as np
import matplotlib.pyplot as plt
from geemap import colormaps

In [7]:
ee.Initialize() 

In [8]:
#collect EE object of buffered wall - exclusive to my google account
# readme file explains creation/uploading shapefile to any other google acc
wall_ee = ee.FeatureCollection('users/Samuelvalman/buffer_350')


In [9]:
#Normalised Difference Vegetation Index (5 dictates landsat 5 band names)
#NIR then Red bands
def getNDVI5(image):
    return image.normalizedDifference(['B4', 'B3'])

In [10]:
#function to get Landsat datasets and filter to period in question, reduce to single image and clip to wall


def NDVI_summer(yr_start, yr_end):
    
    file = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR") \
    .filter(ee.Filter.lt('CLOUD_COVER', 10)) \
    .filter(ee.Filter.calendarRange(yr_start,yr_end,'year')).filter(ee.Filter.calendarRange(6,8,'month')) \
    .filterBounds(wall_ee)
    
    print(file.size().getInfo())
    
    file = file.median()
    file = file.clip(wall_ee)
    
    file = getNDVI5(file)
    return file

In [11]:
#five year ndvi periods 
p1 = NDVI_summer(1985, 1990)
p2 = NDVI_summer(1991, 1995)
p3 = NDVI_summer(1996, 2000)
p4 = NDVI_summer(2001, 2005)
p5 = NDVI_summer(2006, 2010)

7
4
1
2
3


In [13]:
ndviParams = {'palette': colormaps.palettes.ndvi}

Map = geemap.Map()

In [16]:
#add the layers
Map.addLayer(p1, ndviParams, '85-90', shown=True)
Map.addLayer(p2, ndviParams, '91-95', shown=False)
Map.addLayer(p3, ndviParams, '96-00', shown=False)
Map.addLayer(p4, ndviParams, '01-05', shown=False)
Map.addLayer(p5, ndviParams, '06-10', shown=False)


#add the EE_buffer for context 
Map.addLayer(wall_ee, {}, 'buffered', shown=False)

#center on our wall
Map.centerObject(wall_ee, zoom= 10)

In [15]:
Map

Map(center=[54.98234923668915, -2.387573870922565], controls=(WidgetControl(options=['position'], widget=HBox(…

<h1> to download as tif files 

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
fn_p1 = os.path.join(out_dir, 'p1.tif')
fn_p2 = os.path.join(out_dir, 'p2.tif')
fn_p3 = os.path.join(out_dir, 'p3.tif')
fn_p4 = os.path.join(out_dir, 'p4.tif')
fn_p5 = os.path.join(out_dir, 'p5.tif')
region = wall_ee.geometry().bounds().getInfo()

In [ ]:
geemap.ee_export_image(p1, filename=fn_p1, scale=30, region=region, file_per_band=False)
geemap.ee_export_image(p2, filename=fn_p2, scale=30, region=region, file_per_band=False)
geemap.ee_export_image(p3, filename=fn_p3, scale=30, region=region, file_per_band=False)
geemap.ee_export_image(p4, filename=fn_p4, scale=30, region=region, file_per_band=False)
geemap.ee_export_image(p5, filename=fn_p5, scale=30, region=region, file_per_band=False)
